In [1]:
using ITensors
using JLD2

In [2]:
function DMRG(N; g)
    sites = siteinds("S=1/2",N)
    
    H = OpSum()
    for i in 1:N-1
        H += -1,"Sz",i,"Sz",i+1
    end
    for i in 1:N
        H += -g,"Sx",i
    end
    
    MPH = MPO(H,sites)
    ψ = randomMPS(sites, linkdims = 10)
    
    sweeps = Sweeps(40)
    setmaxdim!(sweeps, 10,10,10,20,20,50,100,200,400,400)
    setcutoff!(sweeps, 1E-10)
    setnoise!(sweeps, 1E-6, 1E-7, 1E-8, 0.0)
    E_0, ψ_0 = dmrg(MPH, ψ, sweeps, outputlevel = 0)
    
    return ψ_0
end

DMRG (generic function with 1 method)

In [3]:
function entropy(ψ::MPS)
    L = length(ψ)/2
    b = Int(floor(L))
    
    orthogonalize!(ψ, b)
    U,S,V = svd(ψ[b], (linkind(ψ, b-1), siteind(ψ,b)))
    
    SvN = 0.0
    #normalize!(S)
    for k = 1:dim(S, 1)
        p = S[k,k]^2
        SvN -= p*log(p)
    end
    
    return SvN
end

entropy (generic function with 1 method)

In [4]:
sizes = [61:80;]
S = zeros(Float64, length(sizes))
for i in 1:length(sizes)
    ψ = DMRG(sizes[i], g = 0.5)
    S[i] += entropy(ψ)
end

In [5]:
save_object("Entropy_TFIM.jld2", S)